# RAG Pipeline Demo using Amazon Bedrock

This notebook demonstrates an end-to-end Retrieval-Augmented Generation (RAG) pipeline using Amazon Bedrock, LangChain, and OpenSearch.

In [ ]:
import os
from dotenv import load_dotenv
from app.chunker import chunk_text
from app.embedder import get_bedrock_embedding
from app.retriever import hybrid_search
from app.prompt_templates import build_prompt
from app.bedrock_client import query_bedrock_claude

load_dotenv()


## Step 1: Load and chunk the document

In [ ]:
with open("data/sample_docs/return_policy.pdf", "r") as f:
    document_text = f.read()

chunks = chunk_text(document_text)
print(f"Number of chunks: {len(chunks)}")
print(chunks[:2])  # Show first 2 chunks


## Step 2: Embed the query and run hybrid search

In [ ]:
user_query = "What is the return policy during the holiday season?"
query_embedding = get_bedrock_embedding(user_query)

# Replace with your OpenSearch endpoint
opensearch_url = "https://your-opensearch-endpoint/index/_search"
results = hybrid_search(user_query, query_embedding, opensearch_url)

retrieved_contexts = [hit["_source"]["text"] for hit in results["hits"]["hits"]]
context_text = "\n".join(retrieved_contexts)
print(context_text)


## Step 3: Generate final answer using Claude on Bedrock

In [ ]:
prompt = build_prompt(context_text, user_query)
response = query_bedrock_claude(prompt)
print(response)
